**Jordan Crivella**

*Project 1a*

Topic: Russia and Ukraine

In [82]:
import json
import requests
import requests.exceptions
import pandas as pd

import os
import datetime
import pymongo
import pprint
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import numpy
import mysql.connector
from pandas.io.json import json_normalize
import flatten_json

### Data: UNHCR Data on Refugees https://data2.unhcr.org/en/situations/ukraine ; Kaggle Dataset https://www.kaggle.com/datasets/piterfm/2022-ukraine-russian-war

### 1. Ingest local .csv files, load data frames

In [25]:
#Ingest two local .csv files (russia equiptment and personell losses)
df_person = pd.read_csv('/Users/jordancrivella/Desktop/russia_losses_personnel.csv')
df_eq = pd.read_csv('/Users/jordancrivella/Desktop/russia_losses_equipment.csv')

In [26]:
#View personnel df
df_person.head()

,date,day,personnel,personnel*,POW
0,2022-02-25,2,2800,about,0
1,2022-02-26,3,4300,about,0
2,2022-02-27,4,4500,about,0
3,2022-02-28,5,5300,about,0
4,2022-03-01,6,5710,about,200


In [27]:
#View equiptment df
df_eq.head()

,date,day,aircraft,helicopter,tank,APC,field artillery,MRL,military auto,fuel tank,drone,naval ship,anti-aircraft warfare,special equipment
0,2022-02-25,2,10,7,80,516,49,4,100,60,0,2,0,NaN
1,2022-02-26,3,27,26,146,706,49,4,130,60,2,2,0,NaN
2,2022-02-27,4,27,26,150,706,50,4,130,60,2,2,0,NaN
3,2022-02-28,5,29,29,150,816,74,21,291,60,3,2,5,NaN
4,2022-03-01,6,29,29,198,846,77,24,305,60,3,2,7,NaN


In [28]:
#Join data frames on date to form russia data frame
df_russia = pd.merge(df_person, df_eq, on = 'date', how='outer')

In [29]:
#Preview Russia data frame
df_russia.head()

,date,day_x,personnel,personnel*,POW,day_y,aircraft,helicopter,tank,APC,field artillery,MRL,military auto,fuel tank,drone,naval ship,anti-aircraft warfare,special equipment
0,2022-02-25,2,2800,about,0,2,10,7,80,516,49,4,100,60,0,2,0,NaN
1,2022-02-26,3,4300,about,0,3,27,26,146,706,49,4,130,60,2,2,0,NaN
2,2022-02-27,4,4500,about,0,4,27,26,150,706,50,4,130,60,2,2,0,NaN
3,2022-02-28,5,5300,about,0,5,29,29,150,816,74,21,291,60,3,2,5,NaN
4,2022-03-01,6,5710,about,200,6,29,29,198,846,77,24,305,60,3,2,7,NaN


In [30]:
#Drop unnecessary collumns
drop_cols = ['day_y','personnel*','special equipment']
df_russia.drop(drop_cols, axis=1, inplace=True)

In [31]:
#Preview modified russia data frame
df_russia.head()

,date,day_x,personnel,POW,aircraft,helicopter,tank,APC,field artillery,MRL,military auto,fuel tank,drone,naval ship,anti-aircraft warfare
0,2022-02-25,2,2800,0,10,7,80,516,49,4,100,60,0,2,0
1,2022-02-26,3,4300,0,27,26,146,706,49,4,130,60,2,2,0
2,2022-02-27,4,4500,0,27,26,150,706,50,4,130,60,2,2,0
3,2022-02-28,5,5300,0,29,29,150,816,74,21,291,60,3,2,5
4,2022-03-01,6,5710,200,29,29,198,846,77,24,305,60,3,2,7


### 2.  Dataframe from .csv file to SQL Database

In [42]:
#Assign Connection Variables for the MySQL Server & Databases
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"

user_id = "root"
pwd = "JCrivella1"
dbname = "russia_ukraine"

In [33]:
#Function for Setting Data Into Databases
def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    connection.close()

In [12]:
# Load the Transformed DataFrames into the New 
# Data Warehouse by Creating New Table
exec_sql = f"CREATE DATABASE `{dbname}`;"

conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
sqlEngine.execute(exec_sql) #create db
sqlEngine.execute(f"USE {dbname};") # select new db

In [15]:
db_operation = "insert"

tables = [('russia_losses', df_russia, 'day_x')]

In [16]:
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dbname, dataframe, table_name, primary_key, db_operation)

### 3. File from .csv to .json

In [155]:
# Converting personnel .csv file to .json file on local computer
df_csv = pd.read_csv('/Users/jordancrivella/Desktop/russia_losses_personnel.csv')
df_csv.to_json('/Users/jordancrivella/Desktop/russia_losses_personnel.json')
# Confirmed that it loaded file on my local machine

### 4. Using an API, load .json file to Mongo DB

In [124]:
#Connecting to the MongoDB Instance
host_name = "localhost"
port = "27017"

In [125]:
conn_str = f"mongodb://{host_name}:{port}/"
client = pymongo.MongoClient(conn_str)

In [ ]:
#Define Functions for Getting Data From and Setting Data Into Databases
def get_sql_dataframe(user_id, pwd, host_name, db_name, sql_query):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    conn = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, conn);
    conn.close()
    
    return dframe


def get_mongo_dataframe(user_id, pwd, host_name, port, db_name, collection, query):
    '''Create a connection to MongoDB, with or without authentication credentials'''
    if user_id and pwd:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db_name)
        client = pymongo.MongoClient(mongo_uri)
    else:
        conn_str = f"mongodb://{host_name}:{port}/"
        client = pymongo.MongoClient(conn_str)
    
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    client.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

**From API in .json file to MongoDB**

In [44]:
# Issue a request to an API Endpoint
# Include exception handling (with try/catch error)
def get_api_response(url, response_type):
    try:
        response = requests.get(url)
        response.raise_for_status()
    
    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

    if response_type == 'json':
        result = json.dumps(response.json(), sort_keys=True, indent=4)
    elif response_type == 'dataframe':
        result = pd.json_normalize(response.json())
    else:
        result = "An unhandled error has occurred!"
        
    return result

In [101]:
# Identify response type and url for API
response_type = ['json', 'dataframe']
url = "https://data2.unhcr.org/population/get/sublocation?widget_id=286723&sv_id=54&population_group=5461&forcesublocation=0&fromDate=1900-01-01"

In [102]:
# Data retreival 
json_string = get_api_response(url, response_type[0])
print(json_string)

{
    "data": [
        {
            "admin_level": "Country",
            "centroid_lat": "52.1224",
            "centroid_lon": "19.4013",
            "color": "#3c8dbc",
            "date": "2022-03-24",
            "demography_type": "0",
            "geomaster_id": 10781,
            "geomaster_name": "Poland",
            "households": "0",
            "individuals": "2206119",
            "individuals_type": "2",
            "lat_max": 55.469505777204,
            "lat_min": 48.02154911827,
            "lon_max": 26.4013,
            "lon_min": 12.4013,
            "month": "3",
            "numChildren": "0",
            "population_group_id": "5461",
            "population_groups": [
                {
                    "id": "5461",
                    "name": "Refugee arrivals from Ukraine, any crossing (cumulative)",
                    "pop_accommodation_code": null,
                    "pop_accommodation_name": null,
                    "pop_data_type_code": null,
    

In [103]:
#Create dataframe from API and descrive dataframe
df_refugees = get_api_response(url, response_type[1])

print(df_refugees.shape)
print(df_refugees.columns)

df_refugees.info()

(1, 4)
Index(['situation_view_id', 'situation_view_name',
       'situation_view_description', 'data'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   situation_view_id           1 non-null      object
 1   situation_view_name         1 non-null      object
 2   situation_view_description  1 non-null      object
 3   data                        1 non-null      object
dtypes: object(4)
memory usage: 160.0+ bytes


In [75]:
# Preview data frame and identify issues 
df_refugees

,situation_view_id,situation_view_name,situation_view_description,currentSeries,data.timeseries
0,54,Ukraine Refugee Situation,The escalation of conflict in Ukraine has caus...,1,"[{'data_date': '2022-02-24', 'unix_timestamp':..."


In [76]:
#Drop unnecessary collumns
drop_cols = ['situation_view_id','situation_view_name','situation_view_description']
df_refugees.drop(drop_cols, axis=1, inplace=True)

In [77]:
#Identify column of interest (data column)
df_refugees.head()

,currentSeries,data.timeseries
0,1,"[{'data_date': '2022-02-24', 'unix_timestamp':..."


In [104]:
#Define function to get data from API
def get_api_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
    
    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)
        
    return response.json()

In [105]:
# Extract data from API using function above
json_data = get_api_data(url)
print(json_data)

{'situation_view_id': '54', 'situation_view_name': 'Ukraine Refugee Situation', 'situation_view_description': 'The escalation of conflict in Ukraine has caused destruction of civilian infrastructure and civilian casualties and has forced people to flee their homes seeking safety, protection and assistance. In the first week, more than a million refugees from Ukraine crossed borders into neighbouring countries, and many more are on the move both inside and outside the country. They are in need of protection and support. As the situation continues to unfold, an estimated 4 million people may flee Ukraine. In light of the emergency and paramount humanitarian needs of refugees from Ukraine, an inter-agency regional refugee response is being carried out, in support of refugee-hosting countries’ efforts. The regional refugee response plan brings together UN, NGO and other relevant partners and primarily focuses on supporting the host country governments to ensure safe access to territory for

In [109]:
# Create a new dataframe
# Normalize nested 'data' column to get data of interest on refugees
refugee_data = pd.json_normalize(json_data, record_path=['data'])
refugee_data.head()

,geomaster_name,geomaster_id,admin_level,centroid_lon,centroid_lat,lat_max,lon_max,lat_min,lon_min,source,...,population_groups_concat,population_group_id,individuals_type,demography_type,individuals,households,population_groups,published,numChildren,color
0,Poland,10781,Country,19.4013,52.1224,55.469506,26.401300,48.021549,12.401300,Government,...,"Refugee arrivals from Ukraine, any crossing (c...",5461,2,0,2206119,0,"[{'id': '5461', 'name': 'Refugee arrivals from...",True,0,#3c8dbc
1,Romania,10782,Country,25.3,45.8667,52.866700,32.300000,38.866700,18.300000,Government,...,"Refugee arrivals from Ukraine, any crossing (c...",5461,2,0,572754,0,"[{'id': '5461', 'name': 'Refugee arrivals from...",False,0,#3c8dbc
2,Republic of Moldova,10784,Country,28.4646,47.1976,49.019638,31.685303,45.044273,25.068116,Government,...,"Refugee arrivals from Ukraine, any crossing (c...",5461,2,0,376748,0,"[{'id': '5461', 'name': 'Refugee arrivals from...",True,0,#3c8dbc
3,Hungary,10783,Country,19.4131,47.1672,54.167200,26.588881,43.760061,12.413100,Government,...,"Refugee arrivals from Ukraine, any crossing (c...",5461,2,0,336701,0,"[{'id': '5461', 'name': 'Refugee arrivals from...",False,0,#3c8dbc
4,Russian Federation,10791,Country,38.1556078125,51.488850139616,59.067055,36.958298,54.998769,31.669526,Government,...,"Refugee arrivals from Ukraine, any crossing (c...",5461,2,0,271254,0,"[{'id': '5461', 'name': 'Refugee arrivals from...",False,0,#3c8dbc


In [ ]:
# Drop unnecssary columns 
drop_cols = ['lat_max', 'lat_min','admin_level','centroid_lon', 'centroid_lat', 'individuals_type', 'population_groups', 'numChildren','color','population_groups_concat', 'demography_type', 'households']
refugee_data.drop(drop_cols, axis=1, inplace=True)

In [113]:
# View first 5 rows of refugee data 
refugee_data.head()

,geomaster_name,geomaster_id,lon_max,lon_min,source,date,month,year,population_groups_concat,population_group_id,demography_type,individuals,households,published
0,Poland,10781,26.401300,12.401300,Government,2022-03-24,3,2022,"Refugee arrivals from Ukraine, any crossing (c...",5461,0,2206119,0,True
1,Romania,10782,32.300000,18.300000,Government,2022-03-24,3,2022,"Refugee arrivals from Ukraine, any crossing (c...",5461,0,572754,0,False
2,Republic of Moldova,10784,31.685303,25.068116,Government,2022-03-24,3,2022,"Refugee arrivals from Ukraine, any crossing (c...",5461,0,376748,0,True
3,Hungary,10783,26.588881,12.413100,Government,2022-03-24,3,2022,"Refugee arrivals from Ukraine, any crossing (c...",5461,0,336701,0,False
4,Russian Federation,10791,36.958298,31.669526,Government,2022-03-22,3,2022,"Refugee arrivals from Ukraine, any crossing (c...",5461,0,271254,0,False


In [138]:
# Define connections with Mongo DB
host_name = "localhost"
ports = {"mongo" : 27017, "mysql" : 3306}

user_id = "root"
pwd = "JCrivella1"

dst_dbname = "refugee_data"

In [147]:
#Populate MongoDB with Source Data (russia_ukraine)
port = ports["mongo"]
conn_str = f"mongodb://{host_name}:{port}/"
client = pymongo.MongoClient(conn_str)
db = client[dst_dbname]

In [ ]:
# Insert (run only once)
db.collection.insert_many(refugee_data.to_dict('records'))

In [122]:
#Load Data Frame into Data Warehouse
dataframe = refugee_data
table_name = 'refugee_data'
primary_key = 'geomaster_id'
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

In [148]:
#Verify that 'refugee_data' database was created in MongoDB
client = pymongo.MongoClient(conn_str)
client.list_database_names()

['admin',
 'blog',
 'config',
 'local',
 'northwind_purchasing',
 'refugee_data',
 'russia_ukraine',
 'russia_ukriane']

In [149]:
#Check that collection was inserted
db.list_collection_names()

['collection']